In [3]:
import ctypes
import numpy as np
import math
import time
import cv2
from skimage.color import rgb2lab, lab2rgb

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

class DacQueue:
    '''
    A queue for patterns sent to the dac. 
    Performs smart stitching between patterns.
    '''
    def __init__(self):
        # Dac object for this queue
        self.dac = Dac()
        # The last position of the last pattern (x,y)
        self.last_pos = (0,0)
        # Sample rate of the DAC
        self.dac_rate = 55000
        # Debugging dac rate
        #self.debug_rate = 550
        self.debug_rate = 150
        
    
    def submit(self, pat_pos, pat_col, angular_density=100, debug=False):
        '''
        Submits a new pattern to the dac, transitioning smoothly from the last one.
        Angular density describes how many points per radian should be used to transition
        '''
        # Make the transition
        dist = np.sqrt(np.sum(np.power(self.last_pos-pat_pos[0,:], 2)))
        num_gap_points = int(dist*angular_density)

        gap_pos = np.linspace(self.last_pos, pat_pos[0,:], num_gap_points, dtype=np.float)
        gap_col = np.zeros((num_gap_points, 3), dtype=np.float)
        if(debug):
            gap_col = np.ones_like(gap_col)/4

        # Prep the transition pattern
        gap_points, num_gap_points = self.prep_pattern(gap_pos, gap_col, gap=True)
        
        # Prep the new pattern
        pat_points, num_pat_points = self.prep_pattern(pat_pos, pat_col)
        
        # Send the transition pattern to the dac
        self.write_frames(gap_points, num_gap_points, do_not_loop=True, debug=debug)
        # Send the new pattern to the dac
        self.write_frames(pat_points, num_pat_points, do_not_loop=True, debug=debug)
        # Set the last_position
        self.last_pos = pat_pos[-1, :].copy()
        
    def prep_pattern(self, arr_pos, arr_col, gap=False):
        '''
        1) Scales and color corrects 
        2) Converts from unit space to DAC coordinates
        3) Produces a DAC compatible frame and displays it
        '''
        # Force pass-by-value
        arr_pos = arr_pos.copy()
        arr_col = arr_col.copy()
        # Scale the pattern 
        arr_pos = position_corection(arr_pos)
        # Format the position array (-1.0-1.0) -> int(0-4095)
        arr_pos = (((arr_pos+1)/2.0)*self.dac.xy_max).astype(np.int32)

        # Perform color correction if not gap points
        if(not gap):
            arr_col = color_correction(arr_col)
        # Format the color array (0-1.0) -> int(0-255)
        arr_col = (arr_col*255).astype(np.int32)
        # Fill a heliospoint arr with these values
        num_points = len(arr_pos)

        frameType = HeliosPoint * num_points
        points = frameType()
        # Fill the frame
        for idx in range(num_points):
            points[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                         int(arr_pos[idx, 1]), 
                                         int(arr_col[idx, 0]), 
                                         int(arr_col[idx, 1]),
                                         int(arr_col[idx, 2]),
                                         int(255))
        
        return points, num_points
            
    def write_frames(self, points, num_points, do_not_loop=False, start_immediately=False, debug=False):
        if(num_points < 1):
            return
        # Write the values out 
        status_attempts = 0
        max_attempts = np.inf
        # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
        while(status_attempts < max_attempts and self.dac.HeliosLib.GetStatus(0) != 1):
            status_attempts += 1
            
        # Create flags 
        flags = do_not_loop << 1 | start_immediately << 0
        # Send to DAC
        frame_rate = self.debug_rate if debug else self.dac_rate
        self.dac.HeliosLib.WriteFrame(0, frame_rate, flags, ctypes.pointer(points), num_points)

def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    non_zero = arr_color > 0
    # Scale to the lower cutoff (R, G, B)
    lower_cutoff = np.array([0.25, 0.09, 0.09])
    arr_color = arr_color*(1-lower_cutoff) + lower_cutoff
    
    # Cutoff any zeros to ensure real black
    arr_color *= non_zero
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
def make_circular(arr_pos, arr_color):
    return np.concatenate([arr_pos, arr_pos[::-1, :]]), np.concatenate([arr_color, arr_color[::-1, :]])

def connect_in_space(arr_pos_list, angular_density=100, wait_per=None):
    '''
    Given a list of position arrays, produce a list of array positions that are connected
    '''
    new_arr_pos_list = []
    for i in range(len(arr_pos_list)-1):
        # Get start/end positions for this gap
        pos_start = arr_pos_list[i][-1, :].copy()
        pos_end = arr_pos_list[i+1][ 0, :].copy()
        
        # Make the transition
        dist = np.sqrt(np.sum(np.power(pos_start-pos_end, 2)))
        num_gap_points = int(dist*angular_density)
        gap_pos = np.linspace(pos_start, pos_end, num_gap_points, dtype=np.float)
        
        # If we have a wait period, add it to the end of the gap positions
        if(wait_per):
            rep_bef = np.repeat(pos_start[np.newaxis, :], wait_per, axis=0)
            rep_aft = np.repeat(pos_end[np.newaxis, :], wait_per, axis=0)
            gap_pos = np.concatenate([rep_bef, gap_pos, rep_aft])
            
        
        new_arr_pos_list.append(arr_pos_list[i])
        new_arr_pos_list.append(gap_pos.copy())
    new_arr_pos_list.append(arr_pos_list[-1])
    
    return new_arr_pos_list

def fixed_interp(points, n_interp_points):
    '''
    Expands a list of points to a position array where those 
    points are the anchors in a smooth trajectory. 
    There is a fixed number of interp points between 
    the given points. 
    Inputs: 
        points - (N,2) - list of anchor points
        n_interp_points - number of points between anchors
    '''
    interped_pos_arr = []
    for i in range(len(points)-1):
        p1 = points[i]
        p2 = points[i+1]
        gap_pos = np.linspace(p1, p2, n_interp_points)
        interped_pos_arr += [p1]
        interped_pos_arr += [gap_pos]
    interped_pos_arr += [p2]
    
    # Convert them into one array 
    interped_pos_arr = np.concatenate(interped_pos_arr, axis=0)
    return interped_pos_arr
    


In [4]:
queue = DacQueue()

Found  0 Helios DACs


In [ ]:
# Produce slow circles 
T = 1000
freq = 1.0/100.0
color_arr = np.ones((T,3))/10
start_ang = 0

while(True):
    theta = np.linspace(start_ang, start_ang+2*np.pi, T)
    pos_arr = np.zeros((T, 2))
    pos_arr[:, 0] = np.cos(freq*theta)
    pos_arr[:, 1] = np.sin(freq*theta)
    queue.submit(pos_arr, color_arr)
    start_ang += 2*np.pi

In [ ]:
# Produce colored circles 
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

start = (0.1, 1.0, 0.1)
finish = (1.0, 0.1, 1.0)
smooth = np.linspace(start, finish, T//2)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr)


In [ ]:
# Produce a SINE wave
T = 1000
theta = np.linspace(0, 2*np.pi, T)
freq = 3
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.linspace(-1.0, 1.0, T)
pos_arr[:, 1] = np.sin(theta*freq)

start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr, debug=False)


In [ ]:
# Produce a phase shifting sine wave
T = 600
freq = 4
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.linspace(-1.0, 1.0, T)
start = (0.0, 0.0, 1.0)
finish = (1.0, 1.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(10):
    for phi in np.linspace(0, 2*np.pi, 100):
        theta = np.linspace(phi, phi+2*np.pi, T)

        pos_arr[:, 1] = np.sin(theta*freq)

        queue.submit(pos_arr, color_arr, debug=False)


In [ ]:
# Produce a double SINE wave
T = 1000
theta = np.linspace(0, 4*np.pi, T)
freq = 3
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.concatenate([np.linspace(-1.0, 1.0, T//2), np.linspace(1.0, -1.0, T//2)])
pos_arr[:, 1] = np.sin(theta*freq)

start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4

for i in range(100):
    queue.submit(pos_arr, color_arr, debug=False)


In [ ]:
# Produce a modulated wave
T = 1000
theta = np.linspace(0, 4*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.concatenate([np.linspace(-1.0, 1.0, T//2), np.linspace(1.0, -1.0, T//2)])
start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T)

color_arr = np.concatenate([smooth, smooth[::-1]])/4
for i in range(3):
    for freq in np.linspace(1, 4, 200):
        pos_arr[:, 1] = np.sin(theta*freq)
        queue.submit(pos_arr, color_arr, debug=False)


In [ ]:
# Make a white fan with colored beams in it

beam_dwell = 50
n_beams = 6
beam_col = (0.0, 1.0, 0.0)
fan_col = np.array((1.0, 1.0, 1.0))/4

beam_positions = np.linspace(-1, 1, n_beams)
beams = []
for pos in beam_positions:
    beam = np.zeros((beam_dwell, 2))
    beam[:, 0] = pos
    beams.append(beam.copy())

beam_fan_list = connect_in_space(beams, wait_per=10)
color_arrs = []
for idx,pat in enumerate(beam_fan_list):
    if(idx % 2 == 0):
        color_arrs.append(np.ones((pat.shape[0], 3))*beam_col)
    else:
        color_arrs.append(np.ones((pat.shape[0], 3))*fan_col)
        
pos_arr = np.concatenate(beam_fan_list, axis=0)
col_arr = np.concatenate(color_arrs, axis=0)

pos_arr, col_arr = make_circular(pos_arr, col_arr)

offset_arr = np.zeros_like(pos_arr) 
for i in range(4):
    for y_pos in np.concatenate([np.linspace(-1, 1, 30), np.linspace(1, -1, 60)]):
        offset_arr[:, 1] = y_pos
        queue.submit(pos_arr+offset_arr, col_arr, debug=False)

In [ ]:
# Make a white cone with colored beams in it

beam_dwell = 50
n_beams = 6
beam_col = (0.0, 1.0, 0.0)
fan_col = np.array((1.0, 1.0, 1.0))/4

thetas = np.linspace(0, 2*np.pi, n_beams)
beam_positions = np.stack([np.cos(thetas), np.sin(thetas)], axis=1)
beams = []
for pos in beam_positions:
    beam = np.zeros((beam_dwell, 2))
    beam[:, 0:2] = pos
    beams.append(beam.copy())

beam_fan_list = connect_in_space(beams, wait_per=10)
color_arrs = []
for idx,pat in enumerate(beam_fan_list):
    if(idx % 2 == 0):
        color_arrs.append(np.ones((pat.shape[0], 3))*beam_col)
    else:
        color_arrs.append(np.ones((pat.shape[0], 3))*fan_col)
        
pos_arr = np.concatenate(beam_fan_list, axis=0)
col_arr = np.concatenate(color_arrs, axis=0)

for i in range(100):
    for y_pos in np.concatenate([np.linspace(-1, 1, 2), np.linspace(1, -1, 2)]):
        queue.submit(pos_arr, col_arr, debug=False)

In [ ]:
# Make a white cone with colored beams in it that randomly mutates
beam_dwell = 50
n_beams = 3
beam_col = (0.0, 1.0, 0.0)
fan_col = np.array((1.0, 1.0, 1.0))/8

def normalize_color(color):
    d = math.sqrt(np.sum(color**2))
    return color/d

T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)

thetas = np.linspace(0, 2*np.pi, n_beams+1)
beam_positions = np.stack([np.cos(thetas), np.sin(thetas)], axis=1)
beams = []
for pos in beam_positions:
    beam = np.zeros((beam_dwell, 2))
    beam[:, 0:2] = pos
    beams.append(beam.copy())
beam_fan_list = connect_in_space(beams, wait_per=10)

pattern_size = 0.8
steps = 10000
ang = 0
for step in range(steps):
    ang += 0.01*np.pi
    rot_mat = np.array([[np.cos(ang), -np.sin(ang)], 
                        [np.sin(ang), np.cos(ang)]])
    
    rnd = 0.1*(np.random.rand(3)-0.5)
    beam_col += rnd
    beam_col = np.clip(beam_col, 0, 1)
    beam_col = normalize_color(beam_col)
    color_arrs = []
    for idx,pat in enumerate(beam_fan_list):
        if(idx % 2 == 0):
            color_arrs.append(np.ones((pat.shape[0], 3))*beam_col)
        else:
            color_arrs.append(np.ones((pat.shape[0], 3))*fan_col)

    pattern_size = np.clip(pattern_size + ((np.random.rand(1) > 0.5).astype(np.float32)-0.5)/20, 0.2, 1.0) 
    pos_arr = np.concatenate(beam_fan_list, axis=0)
    col_arr = np.concatenate(color_arrs, axis=0)
    queue.submit(pattern_size*np.matmul(pos_arr, rot_mat), col_arr, debug=False)

In [ ]:
# Circle w/ a spike
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pos_arr = np.zeros((T, 2))
pos_arr[:, 0] = np.cos(theta)
pos_arr[:, 1] = np.sin(theta)
start = (0.0, 0.0, 1.0)
finish = (1.0, 0.0, 1.0)
smooth = np.linspace(start, finish, T//2)
color_arr = np.concatenate([smooth, smooth[::-1]])/4

spike_freq = 20
for i in range(T):
    pos_copy = pos_arr.copy()
    p = (spike_freq*i)%T
    pos_copy[int(p), :] = 0
    queue.submit(pos_copy, color_arr, debug=False)

In [ ]:
np.matmul(np.zeros((T,2)), np.zeros((2,2))).shape

In [ ]:
# Make a smart circle that maps color and radius to some input

# Dynamic range of r
min_r = 0.8
max_r = 1.0
# Correspond to the freq components to visualize
bins = 16
T = 1000
theta = np.linspace(np.pi/2, np.pi/2+2*np.pi, 2*T)

col_arr = np.array((0.5, 0.2, 1.0))[np.newaxis, :]

# Perform the loop 
for i in range(1000):
    # Since we don't have sound input, make it up 
    if(i%200 == 0):
        freq_hist = np.random.rand(bins)*(max_r-min_r) + min_r
        interp_len = int((T-bins)/(bins+1) + 1)
        #r = fixed_interp(freq_hist, n_interp_points=interp_len)[0:T]
        # The array holding the radius of the circle at an angle
        r = np.repeat(freq_hist, T//bins+1)[0:T]
        # Make symmetric
        r = np.concatenate([r, r[::-1]])
    

    
    pos_arr = np.zeros((2*T, 2))
    pos_arr[:, 0] = r*np.cos(theta)
    pos_arr[:, 1] = r*np.sin(theta)
    queue.submit(pos_arr, col_arr*np.ones((2*T, 3)), debug=False)




In [ ]:
freq_hist = np.random.rand(bins)*(max_r-min_r) + min_r
# Find the positions of the components in the circle
r_positions = np.linspace(np.pi/2, np.pi/2+2*np.pi, bins+1, endpoint=False) + 1/(2*(bins+1))
pos = np.stack([np.sin(r_positions), np.sin(r_positions)], axis=1)
fixed_interp(pos, 10)

In [ ]:
np.repeat(freq_hist, 100)[0:T].shape


In [ ]:
# Make a tri-colored circle
# Make a tricolor circle
T = 1000
pos_arr = np.zeros((T, 2))
col_arr = np.zeros((T, 3))

# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
pos_arr[:,0] = np.cos(theta)
pos_arr[:,1] = np.sin(theta)


# Tri color the circle
col_arr[theta < np.pi, 0] = 0
col_arr[theta < np.pi, 1] = 0
col_arr[theta < np.pi, 2] = 0.3
col_arr[theta < (2/3)*np.pi, 0] = 0
col_arr[theta < (2/3)*np.pi, 1] = 0.3
col_arr[theta < (2/3)*np.pi, 2] = 0
col_arr[theta < (1/3)*np.pi, 0] = 0.3
col_arr[theta < (1/3)*np.pi, 1] = 0
col_arr[theta < (1/3)*np.pi, 2] = 0

for i in range(100):
    queue.submit(pos_arr, color_arr)


In [ ]:
scales = np.concatenate([np.linspace(1.0, 0.5, 10), np.linspace(0.5, 1, 20)])

import time
while(True):
    for scale in scales:
        new_pos_arr = scale*pos_arr
        queue.submit(new_pos_arr, col_arr)
    break


In [ ]:
def draw_bitmap(arr_pos, arr_color, dims=(400,400)):
    '''
    Draws a pattern as if we were drawing it on a wall
    '''
    # Copy the arrays so we don't change them 
    arr_pos = arr_pos.copy()
    arr_color = arr_color.copy()
    # Map the positions
    arr_pos[:,0] *= -1
    arr_pos = (((arr_pos+1)/2)*dims).astype(np.int)
    # Map the colors
    arr_color = arr_color[:,::-1]
    T = len(arr_pos)
    img = np.zeros(dims+(3,))
    # Draw points
    for t in range(1, T):
        cv2.line(img, arr_pos[t-1], arr_pos[t], color=arr_color[t-1], thickness=2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
    

In [ ]:
# Make three parallel lines
T_line = 100

arr_pos_list = []
arr_color_list = []
for c_idx, y in enumerate(range(-1, 2, 1)):
    arr_pos_list.append(np.stack([np.linspace(-1, 1, T_line), y*np.ones(T_line)], axis=1))
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())

for i in range(100):
    for arr_pos, arr_color in zip(arr_pos_list, arr_color_list):
        queue.submit(arr_pos, arr_color)

In [ ]:
# Make three concentric circles
T_line = 200
theta_line = np.linspace(0, 2*np.pi, T_line)

arr_pos_list = []
arr_color_list = []
for c_idx, scale in enumerate([0.9, 0.95, 1]):
    arr_pos_list.append(np.stack([np.cos(theta_line), np.sin(theta_line)], axis=1)/scale)
    arr_color = np.zeros((T_line, 3))
    arr_color[:, c_idx] = np.linspace(0, 1, T_line)
    arr_color_list.append(arr_color.copy())

for i in range(1000):
    for arr_pos, arr_color in zip(arr_pos_list, arr_color_list):
        queue.submit(arr_pos, arr_color)

In [ ]:
draw_bitmap(arr_pos, arr_color)

In [ ]:
# make a sine wave
T = 500
freq = 4
theta_line = np.linspace(0, 2*np.pi, T)
arr_pos = np.stack([np.linspace(-1, 1, T), np.sin(freq*theta_line)], axis=1)
#arr_color = np.ones((T, 3))
arr_color = np.linspace((0.1, 1.0, 0.3), (1.0, 0.0, 0.7), T)
for i in range(100):
    queue.submit(arr_pos, arr_color, debug=False)

In [ ]:
print(lab2rgb((0,0,0)))
print(rgb2lab((0.10,0.10,0.255)))
print(rgb2lab((0.5,1,1)))
print('this: ', lab2rgb((10.0,-100.0, 10.0)))



l_lims = (0, 100)
a_lims = (-100, 100)
b_lims = (-100, 100)

w = a_lims[1] - a_lims[0] 
h = b_lims[1] - b_lims[0] 
img = np.zeros((h, w, 3), np.float32)

for a_idx, a in enumerate(np.linspace(*a_lims, h)):
    for b_idx, b in enumerate(np.linspace(*b_lims, w)):
        img[a_idx, b_idx, :] = lab2rgb((90.0, float(a), float(b)))

cv2.imshow('frame', img)
cv2.waitKey(0)

In [ ]:
good = []
bad = []
for l in range(-100, 100):
    print(l)
    for a in range(-100, 100): 
        for b in range(-100, 100):
            try:
                rgb = lab2rgb(np.array([l, a, b]))
                good += [(l,a,b)]
            except:
                bad += [(l,a,b)]



In [9]:
start = -127
stop = 128
num_points = stop - start + 1
#num_points = (stop - start + 1)//3
l = np.linspace(0, 100, num_points)
a = np.linspace(start, stop, num_points)
b = np.linspace(start, stop, num_points)
#np.repeat(l[np.newaxis, repeats=10, axis=1)
#lab = np.stack([l[:, np.newaxis, np.newaxis], a[np.newaxis, :, np.newaxis], b[np.newaxis, np.newaxis, :]])

l, a, b = np.meshgrid(l, a, b, indexing='ij')
lab = np.stack([l, a, b], axis=-1)
lab[1, 2, 3]

rgb = lab2rgb(lab)
print(np.min(rgb), np.max(rgb), np.mean(rgb))


/home/bij/anaconda3/envs/Laser/lib/python3.8/site-packages/skimage/color/colorconv.py:1128: UserWarning: Color data out of range: Z < 0 in 1940992 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


0.0 1.0 0.4239979685029926


In [13]:
while(True):
    for lum in np.concatenate([np.arange(num_points), np.arange(num_points)[::-1]]):
        frame = rgb[lum, :, :, :]
        print('Lum', lum, np.min(frame), np.max(frame), np.mean(frame))
        cv2.imshow('frame', cv2.resize(frame, None, fx=2, fy=2))
        cv2.waitKey(1)

Lum 0 0.0 0.7578495556207594 0.12083769660173072
Lum 1 0.0 0.7622663711612824 0.12247870160369327
Lum 2 0.0 0.7666891448475324 0.12413461717475816
Lum 3 0.0 0.7711178446141438 0.12580463333765346
Lum 4 0.0 0.7755524388315317 0.1274885957734014
Lum 5 0.0 0.7799928962971352 0.12918604776259612
Lum 6 0.0 0.784439186226881 0.13089654546384583
Lum 7 0.0 0.788891278246857 0.1326201444985962
Lum 8 0.0 0.79334914238519 0.13435698971394086
Lum 9 0.0 0.797812749064126 0.1361062645855174
Lum 10 0.0 0.8022820690923003 0.13786736705121527
Lum 11 0.0 0.8067570736571987 0.13963994279565686
Lum 12 0.0 0.8112377343178024 0.14142274721982262
Lum 13 0.0 0.8157240229974071 0.14321417993348717
Lum 14 0.0 0.8202159119766196 0.14501308917717634
Lum 15 0.0 0.8247133738865196 0.14681843490079463
Lum 16 0.0 0.8292163817019864 0.1486287172382809
Lum 17 0.0 0.8337249087351839 0.15044317566919024
Lum 18 0.0 0.8382389286292014 0.1522612090219412
Lum 19 0.0 0.8427584153518429 0.15408302015605163
Lum 20 0.0 0.8472833

Lum 216 0.0 1.0 0.6599728504523829
Lum 217 0.0 1.0 0.6625652275099128
Lum 218 0.0 1.0 0.6651300760185976
Lum 219 0.0 1.0 0.6676709088207765
Lum 220 0.0 1.0 0.6701901448955039
Lum 221 0.0 1.0 0.6726847516542303
Lum 222 0.0 1.0 0.6751534227133323
Lum 223 0.0 1.0 0.677606854926707
Lum 224 0.0 1.0 0.6800356050640067
Lum 225 0.0 1.0 0.6824428514252564
Lum 226 0.0 1.0 0.6848304643299913
Lum 227 0.0 1.0 0.6872015084921917
Lum 228 0.0 1.0 0.689549108054364
Lum 229 0.0 1.0 0.6918773178980269
Lum 230 0.0 1.0 0.694191279042891
Lum 231 0.0 1.0 0.6964841208037412
Lum 232 0.0 1.0 0.6987567470238113
Lum 233 0.0 1.0 0.701014121133828
Lum 234 0.0 1.0 0.7032550130524163
Lum 235 0.0 1.0 0.7054739615758021
Lum 236 0.0 1.0 0.707677064066433
Lum 237 0.0 1.0 0.7098645984022913
Lum 238 0.0 1.0 0.712033419106426
Lum 239 0.0 1.0 0.7141806057202986
Lum 240 0.0 1.0 0.7163160604266742
Lum 241 0.0 1.0 0.7184304451523246
Lum 242 0.0 1.0 0.7205247135717016
Lum 243 0.0 1.0 0.7226003253467524
Lum 244 0.0 1.0 0.72465583

Lum 49 0.0 0.9801502934690732 0.2161466841996674
Lum 48 0.0 0.9755190290874428 0.21384715475100788
Lum 47 0.0 0.970891193985114 0.2115624902375817
Lum 46 0.0 0.9662667999485041 0.2092925702987055
Lum 45 0.0 0.9616458589226092 0.20703806442992145
Lum 44 0.0 0.9570283830145546 0.20479862811747426
Lum 43 0.0 0.952414384497246 0.2025745389090604
Lum 42 0.0 0.9478038758131168 0.2003661238474801
Lum 41 0.0 0.943196869577983 0.19817279035057359
Lum 40 0.0 0.9385933785850059 0.19599477443444482
Lum 39 0.0 0.9339934158087647 0.1938319472675821
Lum 38 0.0 0.9293969944094433 0.19168546673215023
Lum 37 0.0 0.9248041277371375 0.18955374385342616
Lum 36 0.0 0.9202148293362823 0.18743779027628785
Lum 35 0.0 0.9156291129502058 0.18533811669408462
Lum 34 0.0 0.9110469925258147 0.183254425384382
Lum 33 0.0 0.9064684822184113 0.18118664311268398
Lum 32 0.0 0.9018935963966523 0.1791359068322257
Lum 31 0.0 0.8973223496476498 0.1771012681440897
Lum 30 0.0 0.8927547567822194 0.1750841769932107
Lum 29 0.0 0.8

Lum 172 0.0 1.0 0.5398186924872704
Lum 173 0.0 1.0 0.5425415567913106
Lum 174 0.0 1.0 0.5452657534181146
Lum 175 0.0 1.0 0.5479927026900802
Lum 176 0.0 1.0 0.5507185999366346
Lum 177 0.0 1.0 0.5534455633030863
Lum 178 0.0 1.0 0.5561739868537957
Lum 179 0.0 1.0 0.5589049173441204
Lum 180 0.0 1.0 0.5616339270197402
Lum 181 0.0 1.0 0.5643654510468491
Lum 182 0.0 1.0 0.567098674324046
Lum 183 0.0 1.0 0.5698328646118096
Lum 184 0.0 1.0 0.5725655963584618
Lum 185 0.0 1.0 0.5753028493700345
Lum 186 0.0 1.0 0.5780407979077677
Lum 187 0.0 1.0 0.5807774004386504
Lum 188 0.0 1.0 0.5835174724486382
Lum 189 0.0 1.0 0.5862587664298154
Lum 190 0.0 1.0 0.5890014381850643
Lum 191 0.0 1.0 0.5917424261678134
Lum 192 0.0 1.0 0.5944897829527057
Lum 193 0.0 1.0 0.5972356066030646
Lum 194 0.0 1.0 0.5999823283103948
Lum 195 0.0 1.0 0.6027314462799366
Lum 196 0.0 1.0 0.6054826324156309
Lum 197 0.0 1.0 0.608233739288302
Lum 198 0.0 1.0 0.6109841538057407
Lum 199 0.0 1.0 0.6137397920594364
Lum 200 0.0 1.0 0.6164

Lum 82 0.0 1.0 0.2975051145559301
Lum 81 0.0 1.0 0.2949199302359818
Lum 80 0.0 1.0 0.292340298583266
Lum 79 0.0 1.0 0.2897675639128994
Lum 78 0.0 1.0 0.2872017378378497
Lum 77 0.0 1.0 0.2846429774436319
Lum 76 0.0 1.0 0.28209066380483727
Lum 75 0.0 1.0 0.27954518545257495
Lum 74 0.0 1.0 0.2770071492408986
Lum 73 0.0 1.0 0.2744759439139361
Lum 72 0.0 1.0 0.2719526166632163
Lum 71 0.0 1.0 0.26943501837606953
Lum 70 0.0 1.0 0.26692581833809165
Lum 69 0.0 1.0 0.2644241118292758
Lum 68 0.0 1.0 0.2619301766238942
Lum 67 0.0 1.0 0.25944241404413987
Lum 66 0.0 1.0 0.256963674065841
Lum 65 0.0 1.0 0.2544923551637201
Lum 64 0.0 1.0 0.25202908057703716
Lum 63 0.0 1.0 0.2495736701192394
Lum 62 0.0 1.0 0.24712597425647398
Lum 61 0.0 1.0 0.24468668498602586
Lum 60 0.0 1.0 0.24225630205102186
Lum 59 0.0 1.0 0.23983404757765836
Lum 58 0.0 1.0 0.23742020347941858
Lum 57 0.0 1.0 0.23501565192731724
Lum 56 0.0 1.0 0.2326186427111486
Lum 55 0.0 1.0 0.2302323355029944
Lum 54 0.0 1.0 0.22785470872267877
Lum

Lum 140 0.0 1.0 0.4530258356134169
Lum 141 0.0 1.0 0.4557305166890835
Lum 142 0.0 1.0 0.45843503016751247
Lum 143 0.0 1.0 0.4611391610849565
Lum 144 0.0 1.0 0.46384585465234224
Lum 145 0.0 1.0 0.46655104867737873
Lum 146 0.0 1.0 0.4692566256779321
Lum 147 0.0 1.0 0.47196327299356783
Lum 148 0.0 1.0 0.4746710838682437
Lum 149 0.0 1.0 0.4773771638881545
Lum 150 0.0 1.0 0.4800850495723659
Lum 151 0.0 1.0 0.4827940605490008
Lum 152 0.0 1.0 0.4855025606430711
Lum 153 0.0 1.0 0.48821134223274504
Lum 154 0.0 1.0 0.490921753263948
Lum 155 0.0 1.0 0.4936331535538076
Lum 156 0.0 1.0 0.4963428530718586
Lum 157 0.0 1.0 0.49905545351402036
Lum 158 0.0 1.0 0.501768169037018
Lum 159 0.0 1.0 0.5044814813610546
Lum 160 0.0 1.0 0.5071938864769114
Lum 161 0.0 1.0 0.5099098253449657
Lum 162 0.0 1.0 0.5126248508364033
Lum 163 0.0 1.0 0.515339746507487
Lum 164 0.0 1.0 0.518056918082673
Lum 165 0.0 1.0 0.5207744327669301
Lum 166 0.0 1.0 0.5234929059166248
Lum 167 0.0 1.0 0.5262099964276219
Lum 168 0.0 1.0 0.

KeyboardInterrupt: 

In [1]:
# RGB Comparison
start = 0
stop = 1
num_points = 400

r = np.linspace(start, stop, num_points)
g = np.linspace(start, stop, num_points)


r, g = np.meshgrid(r, g, indexing='ij')
b = 0.5*np.zeros((400, 400))
print(r.shape)
rg = np.stack([r, g, b], axis=-1)
print(rg.shape)
cv2.imshow('frame', cv2.resize(rg[:, :, :], None, fx=2, fy=2))
cv2.waitKey(0)

NameError: name 'np' is not defined

In [ ]:
# Circles in the CIELAB Space
# UNFINISHED - see this for documentation: https://github.com/scikit-image/scikit-image/blob/main/skimage/color/colorconv.py#L1142-L1183
# Range is here? :https://stackoverflow.com/questions/25294141/cielab-color-range-for-scikit-image
scales = np.concatenate([np.linspace(1, 0.1, 200), np.linspace(0.1, 1, 200)])

T = 400
n_scales = 10

# Set Xs and Ys
theta = np.linspace(0, 2*np.pi, T)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)

arr_col = np.ones((T, 3))

# Set the colors 
arr_cols = []
for scale in scales:
    arr_col_s = arr_col*lab[]
    

for i in range(1000):
    for scale, arr_col in zip(scales, arr_cols):
        queue.submit(arr_pos*scale, arr_col)

    
    

In [ ]:
img = np.zeros((10000, 10000, 2))
for row in range(10000):
    for col in range(10000):
        img[row, col] = row, col

In [ ]:
img.shape

In [ ]:
N = 10
x = np.linspace(0, N-1, N)
y = np.linspace(0, N-1, N)
x = np.stack([x]*N, axis=1)
y = np.stack([y]*N, axis=0)
stacked = np.stack([x, y], axis=-1)

In [ ]:
print(stacked)

In [ ]:
stacked[9, 1]

In [ ]:
# Random ellipses
T = 500
arr_col = np.ones((T, 3))
while(True):
    a = np.random.rand(1)
    b = np.random.rand(1)
    theta = np.linspace(0, 2*np.pi, T)
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = a*np.cos(theta)
    arr_pos[:,1] = b*np.sin(theta)
    col_gate = np.repeat(np.random.rand(T//10, 3) > 0.2, 10, axis=0)
    for i in range(20):
        queue.submit(arr_pos, arr_col*col_gate)

    

In [ ]:
# Interpolated ellipses
T = 500
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

a_old = 1.0
b_old = 1.0
c_old = (1.0, 1.0, 1.0)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)
accel_max = math.sqrt((1-0)**2 + (1-0)**2)

T_trans = 5

while(True):
    a = np.random.rand(1)[0]
    b = np.random.rand(1)[0]
    c = np.random.rand(3)
    
    #col_gate = np.repeat(np.random.rand(T//10, 3) > 0.5, 10, axis=0)
    col_gate = 1
    ell_interp = np.linspace((a_old, b_old), (a, b), T_trans)
    col_interp = np.linspace(c_old, c, T_trans, axis=0)
    accel = math.sqrt((a_old-a)**2 + (b_old-b)**2)
    accel_norm = accel/accel_max

    for ell_inter, c_inter in zip(ell_interp, col_interp):
        queue.submit((ell_inter*arr_pos), np.repeat(c_inter[np.newaxis, :], T, axis=0))
        
    a_old = a
    b_old = b
    c_old = c

    

In [ ]:
# Interpolated ellipses with beamifier (dweller)
T = 500
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

a_old = 1.0
b_old = 1.0
c_old = (1.0, 1.0, 1.0)
arr_pos = np.zeros((T, 2))
arr_pos[:,0] = np.cos(theta)
arr_pos[:,1] = np.sin(theta)
accel_max = math.sqrt((1-0)**2 + (1-0)**2)

T_trans = 8

while(True):
    a = np.random.rand(1)[0]
    b = np.random.rand(1)[0]
    c = np.random.rand(3)
    
    ell_interp = np.linspace((a_old, b_old), (a, b), T_trans)
    col_interp = np.linspace(c_old, c, T_trans, axis=0)
    accel = math.sqrt((a_old-a)**2 + (b_old-b)**2)
    accel_norm = accel/accel_max

    
    for ell_inter, c_inter in zip(ell_interp, col_interp):
        arr_pos_inter = ell_inter*arr_pos
        c_arr = np.repeat(c_inter[np.newaxis, :], T, axis=0)
        #arr_pos_beam = beamify(arr_pos_inter, dwell=4)
        #queue.submit(arr_pos_beam, c_arr)
        queue.submit(arr_pos_inter, c_arr)
        
    a_old = a
    b_old = b
    c_old = c

    

In [ ]:
# Beamify
x = np.linspace(0, 1, 100)
x = np.repeat(x[::10], 10)

In [ ]:
def beamify(arr_pos, num_beams=4, dwell=20):
    '''
    NOT DONE - inserts spots in the trajectory where the beam stops and makes a solid beam
    '''
    T = arr_pos.shape[0]
    beam_idxs = np.linspace(0, T, num_points+2)[1:-1].astype(np.int)
    np.insert(arr_pos)
    return arr_pos

In [ ]:
x = np.zeros(101)
num_points = 4
beam_idxs = np.linspace(0, x.shape[0], num_points+2)[1:-1].astype(np.int)

In [ ]:
beam_idxs

In [ ]:
def signer(f, T, phase=0.0):
    '''
    Produces spaced out points from 0 to 1 whose distance between 
    consecutive points is sinusoidal 
    Input:  f - frequency 
            T - number of samples
            phase - phase shift (0, 1)
    Returns: x - positions on the line
             dx - the derivative of the function of x
    '''
    # Convert to radians
    f = f*2*np.pi
    # Generate the baseline 
    t = np.linspace(0.0+phase, 1.0+phase, T)
    # Find points 
    x = (f*t - np.cos(f*t) + 1)/f
    # Calculate derivative
    dx = (np.sin(f*t) + 1)/2
    
    return x, dx

# STILL UNDER DEVELOPMENT
def triangle(theta, alpha, sincos='sin'):
    '''
    Returns a wave between a sine wave and triangle wave depending on alpha
    alpha = 0 - sine wave
    alpha = 1 - triangle wave
    '''
    if(sincos == 'sin'):
        ps = 0
    elif(sincos == 'cos'):
        ps = np.pi/2
    return alpha*2*np.arcsin(np.sin(theta + ps))/np.pi + (1-alpha)*np.sin(theta + ps)
    
def ellipser(scale_x=1, scale_y=1, f_x=1, f_y=1, f_s=1000, phase_s=0, sintriang=0, colors=[(1,1,1)], T=500):
    '''
    Generates a pattern that is a function of ellipses 
    with parameters that can be interpolated between
    '''
    theta_norm, theta_dx = signer(f=f_s, phase=phase_s, T=T)
    theta = theta_norm*2*np.pi
    arr_pos = np.zeros((T, 2))
    arr_pos[:,0] = scale_x*triangle(f_x*theta, sintriang, 'sin')
    arr_pos[:,1] = scale_y*triangle(f_y*theta, sintriang, 'cos')
    
    # Color the pattern
    arr_col = np.ones((T, 3), np.float32)
    if(len(colors) == 1):
        # All one color
        arr_col[:, :] = colors[0][np.newaxis, :]
    if(len(colors) == 2):
        # Color based on speed of laser
        arr_col[theta_dx > 0.1, :] = colors[0]
        arr_col[theta_dx <= 0.1, :] = colors[1]
    
    return arr_pos, arr_col


In [ ]:
# Interpolated ellipses
T = 750
arr_col = np.ones((T, 3))
theta = np.linspace(0, 2*np.pi, T)

min_vals = np.array([0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
max_vals = np.array([1.0, 0.4, 2.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
pat_point_old = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals


T_trans = 150

while(True):
    pat_point = (max_vals-min_vals)*np.random.rand(len(max_vals)) + min_vals
    ell_interp = np.linspace(pat_point_old, pat_point, T_trans)
    for (scale_x, scale_y, freq_x, freq_y, f_s, phase, sintriang, r1, g1, b1, r2, g2, b2) in ell_interp:
        # Transform the distribution of f_s to be less uniform
        f_s = 40*np.power(f_s, 6)
        #TODO - transform sintriang to have a bathtub curve
        
        # Get the array
        arr_pos, arr_col = ellipser(scale_x=scale_x, 
                                    scale_y=scale_y, 
                                    f_x=freq_x, 
                                    f_y=freq_y, 
                                    f_s=f_s, 
                                    phase_s=phase,
                                    sintriang=sintriang, 
                                    colors=[(r1,g1,b1), (r2,g2,b2)],
                                    T=T)
        
        queue.submit(arr_pos, arr_col)
        
    pat_point_old = pat_point

In [ ]:
T = 1000
arr_col_og = np.ones((T, 3))
x, dx = signer(f=10, T=T, shift=0.75)
theta = 2*np.pi*x
arr_pos = np.zeros((T, 2))
arr_pos[:, 0] = np.cos(theta)
arr_pos[:, 1] = np.sin(theta)


while(True):
    for thresh in np.linspace(0, 1, 100):
        arr_col = arr_col_og*(thresh < (1-dx[:,np.newaxis]))
        queue.submit(arr_pos, arr_col)
